In [4]:
import pydeck as pdk
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('./phantom_grouped.csv')

In [6]:
len(data)

314219

In [8]:
data.head()

,count,end_lat,end_lon,next_lat,next_lon
0,15678,40.726218,-73.983799,40.729387,-73.977724
1,12721,40.756405,-73.990026,40.755136,-73.986580
2,8305,40.729387,-73.977724,40.726218,-73.983799
3,8231,40.726218,-73.983799,40.733143,-73.975739
4,6378,40.729236,-73.990868,40.726218,-73.983799


In [21]:
sorted_data = data.sort_values(by='count',ascending = False)

In [58]:
GREEN_RGB = [0, 255, 0, 80]
RED_RGB = [240, 100, 0, 80]
arc_layer = pdk.Layer(
    "ArcLayer",
    data = sorted_data[:1000],
    get_source_position=['end_lon','end_lat'],
    get_target_position = ['next_lon','next_lat'],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=GREEN_RGB,
    pickable=True,
    auto_highlight=True,
    get_width = 'count/700')

view_state = pdk.ViewState(latitude = 40.778786,longitude = -73.974785, bearing=45, pitch=50, zoom=11,)
# latitude = 40.778786,longitude = -73.974785

TOOLTIP_TEXT = {"html": "{count} bikes <br /> Home of commuter in red; work location in green"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT,map_style = 'light')
r.to_html("arc_layer.html")

In [33]:
deck = pdk.Deck(layers = arc_layer,map_style='carto_positron')
deck.show()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"lat…